# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y[:5]

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.copy()
X.drop("credit_ranking", axis=1, inplace=True)

# Review the features DataFrame
X.head()


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset & Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features


11

In [56]:
# Define the number of hidden nodes for the first hidden layer
first_hidden_nodes = 6

# Define the number of hidden nodes for the second hidden layer
second_hidden_nodes = 3

# Define the number of neurons in the output layer
number_output_neurons = 1

In [57]:
# Create the Sequential model instance
nn = Sequential()

# Add the first hidden layer
nn.add(Dense(units=first_hidden_nodes, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer
nn.add(Dense(units=second_hidden_nodes, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))

In [58]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 6)                 72        
                                                                 
 dense_10 (Dense)            (None, 3)                 21        
                                                                 
 dense_11 (Dense)            (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [59]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [60]:
# Fit the model using 50 epochs and the training data
model_1 = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50


38/38 [==============================] - 0s 521us/step - loss: 0.7007 - accuracy: 0.5146
Epoch 2/50
38/38 [==============================] - 0s 427us/step - loss: 0.6776 - accuracy: 0.5855
Epoch 3/50
38/38 [==============================] - 0s 390us/step - loss: 0.6639 - accuracy: 0.6038
Epoch 4/50
38/38 [==============================] - 0s 390us/step - loss: 0.6488 - accuracy: 0.6214
Epoch 5/50
38/38 [==============================] - 0s 382us/step - loss: 0.6328 - accuracy: 0.6314
Epoch 6/50
38/38 [==============================] - 0s 376us/step - loss: 0.6158 - accuracy: 0.6522
Epoch 7/50
38/38 [==============================] - 0s 385us/step - loss: 0.6007 - accuracy: 0.6606
Epoch 8/50
38/38 [==============================] - 0s 402us/step - loss: 0.5865 - accuracy: 0.6706
Epoch 9/50
38/38 [==============================] - 0s 383us/step - loss: 0.5749 - accuracy: 0.6764
Epoch 10/50
38/38 [==============================] - 0s 397us/step - loss: 0.5649 - accuracy: 0.7048
Epoch 11/5

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [61]:
#scale the x_test
X_test_scaled = X_scaler.transform(X_test)

In [62]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 1.8568 - accuracy: 0.6200 - 59ms/epoch - 5ms/step
Loss: 1.856788992881775, Accuracy: 0.6200000047683716


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [63]:
# Set the model's file path
file_path = Path("student_loan_model.keras")

# Export your model to a keras file
nn.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [64]:
# Set the model's file path
file_path = Path("student_loan_model.keras")

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [65]:
# Make predictions with the test data
y_pred = nn_imported.predict(X_test).ravel()

# Display a sample of the predictions
y_pred[:5]

13/13 [==============================] - 0s 352us/step


array([0.8862322 , 0.30844826, 0.9999775 , 0.9256492 , 0.99996084],
      dtype=float32)

In [66]:
print(y_pred.shape)


(400,)


In [67]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions = nn_imported.predict(X_test).ravel()

results = pd.DataFrame(predictions, columns=['Predicted'])
results["Predicted"] = results["Predicted"].apply(lambda x: int(round(x)))
results


 1/13 [=>............................] - ETA: 0s

13/13 [==============================] - 0s 401us/step


,Predicted
0,1
1,0
2,1
3,1
4,1
...,...
395,1
396,1
397,1
398,0


### Step 4: Display a classification report with the y test data and predictions

In [68]:
# Print the classification report with the y test data and predictions

predicted_labels = results['Predicted'].values  # Extract the predictions from the DataFrame

# Print the classification report using 'y_test' and the predicted labels
print(classification_report(y_test, predicted_labels))


              precision    recall  f1-score   support

           0       0.70      0.34      0.45       188
           1       0.60      0.87      0.71       212

    accuracy                           0.62       400
   macro avg       0.65      0.60      0.58       400
weighted avg       0.65      0.62      0.59       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

Answer: Data that would be relevant to a student loan recommendation system could vary widely. I would like to information regarding the student's employment status, income level, credit score, repayment history, current debts, field of study and degree completion date, and post-graduation income data. All of this data is directly indicative of the student's potential ability to repay their loan after graduation. 

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

Answer: My model would use a content-based filtering, because it can utilize the specific attributes of each individual student to recommend financing that fits their personal profile. 

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

Answer: The primary challenge would be data privacy and security. This data is highly sensitive and personal, and would need to be protected as such. This would help with trust of the system and legal compliance.
Another challenge would be monitoring the model for any bias - ensuring that it treats all students fairly without discrimination based on data that doesn't directly impact their ability to receive or pay back financing. 

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

